## Importing libraries; initialising logging and reading data

In [1]:
import pandas as pd
import numpy as np
import logging
import re

In [2]:
# ascii / hex code checking

ascii_code = 188
character = chr(ascii_code)
print(character)

hex_boi_char = bytes.fromhex("20").decode('utf-8')
print(hex_boi_char)

¼
 


In [3]:
# Set up logging

logging.basicConfig(
    filename='data_cleaning.log',  # Log file name
    level=logging.INFO,             # Log level
    format='%(asctime)s - %(levelname)s - %(message)s'
)

In [4]:
# read in Numunit

file_path = '2024_03_numunit_withOMOPtarget_synonyms.txt'
data = pd.read_csv(file_path)

# Ensure there are no NaN values in the 'description' column
data.dropna(subset=['description'], inplace=True)


# initialise description_clean column
data['description_clean'] = data['description']

## Handling synthetic training data

In [5]:
# read in suggested changes from Zara to add to synthetic training data

# Path to the CSV file
review_file_path = "R:/Projects/CPRD_OMOPUnits/Results/Review 07.10.2024/SAMPLE_2PC_NEWMAPS_ZCReview.csv"
review_date = review_file_path[35:52]

# Read the CSV file into a DataFrame
review_data = pd.read_csv(review_file_path)

# Create a dictionary to store the non-blank values from 'ZC_ReviewCat' with 'description' as the key
zc_reviewcat_dict = {}
for index, row in review_data.iterrows():
    description = row['description']
    zc_reviewcat = row['ZC_ReviewCat']
    if pd.notna(zc_reviewcat):  # Check if the value is not NaN
        zc_reviewcat_dict[description] = zc_reviewcat

        
dict_output_file_path = "synthetic_train/additions.txt"

# Write the contents of the dictionary to the text file
with open(dict_output_file_path, 'w') as file:
    for description, review_cat in zc_reviewcat_dict.items():
        file.write(f"{description}: {review_cat}\n")

In [6]:
# create synthetic training data to add to Numunit

# Define the column names
synth_columns = ['numunitid', 'obsval', 'source_code_description', 'description', 'synonyms', 'description_clean']

# Create an empty DataFrame with the specified columns
synth_train_data = pd.DataFrame(columns=synth_columns)

# Provided values for 'description_clean'
new_entries_file_path = "synthetic_train/additions.txt"
new_entries = {}

with open(new_entries_file_path, 'r') as file:
    for line in file:
        # Strip whitespace and split the line into key and value
        key_value = line.strip().split(': ')
        if len(key_value) == 2:  # Ensure there are exactly two parts
            key = key_value[0]
            value = key_value[1]
            new_entries[key] = value



            
# Create a list of dictionaries to populate the DataFrame
rows_to_add = [{'description_clean': description, 'source_code_description': source_code} for description, source_code in new_entries.items()]

# Convert the list of dictionaries into a DataFrame
rows_to_add_df = pd.DataFrame(rows_to_add)

# Concatenate the new DataFrame with the existing 'df' DataFrame
synth_train_data = pd.concat([synth_train_data, rows_to_add_df], ignore_index=True)
    
synth_train_data.sample(n=10)

,numunitid,obsval,source_code_description,description,synonyms,description_clean
37,NaN,NaN,per week,NaN,NaN,OUNCE week
22,NaN,NaN,unit of white blood cell analysis,NaN,NaN,WBC 10.42
0,NaN,NaN,unit of white blood cell analysis,NaN,NaN,WBC 9.01
34,NaN,NaN,score,NaN,NaN,night score
33,NaN,NaN,million per microliter,NaN,NaN,x 10 6/ul
4,NaN,NaN,unit of white blood cell analysis,NaN,NaN,WBC 4.72
14,NaN,NaN,unit of white blood cell analysis,NaN,NaN,WBC 12.34
28,NaN,NaN,per month,NaN,NaN,UNITS A MONTH
12,NaN,NaN,unit of white blood cell analysis,NaN,NaN,high WBC 12.78
32,NaN,NaN,not a valid unit,NaN,NaN,12 hours post pain


In [7]:
# Append the synthetic trainign data to Numunit
data = pd.concat([data, synth_train_data], ignore_index=True)
data.sample(10)

,numunitid,obsval,source_code_description,description,synonyms,description_clean
8769,15081,6,billion per liter,x10^9/l(58.0%),"10E9/L, 10^6/mm3, 10^3/uL, 10*9/l, 10^3/mm3, 1...",x10^9/l(58.0%)
1673,730,923,NaN,Urine glucose test = trace,NaN,Urine glucose test = trace
11215,24928,3,NaN,32/35,NaN,32/35
1835,1420,660,NaN,g/collection,NaN,g/collection
14797,14824,2,billion per liter,x10^9/l(61.3%),"10E9/L, 10^6/mm3, 10^3/uL, 10*9/l, 10^3/mm3, 1...",x10^9/l(61.3%)
4994,9972,29,NaN,10*3 cells/L,NaN,10*3 cells/L
12568,17663,2,NaN,/week1,NaN,/week1
3524,5944,90,NaN,Faeces sent for examination,NaN,Faeces sent for examination
3835,4480,71,NaN,mmol/m creat,NaN,mmol/m creat
19849,28540,1,NaN,mg/m2,NaN,mg/m2


## Non-complex character cleaning

In [8]:
# defining character lists and dictionaries

ascii_list = {
    181: 'u',
    186: 'deg',
    176: 'deg',
    185: '1',
    178: '2',
    179: '3',
    195: ' ',
    194: ' ',
    33: ' ',
    36: ' ',
    44: ' ',
    59: ' ',
    95: ' ',
    123: ' ',
    125: ' ',
    40: ' ',
    41: ' ',
    93: ' ',
    130: ' ',
    131: ' ',
    166: ' ',
    172: ' ',
    191: ' ',
    198: ' ',
    188: ' fraction one quarter ',
    163: ' pounds ',
    92: ' per ',
    64: ' at ',
    62: ' greater than ',
    61: ' equals ',
    60: ' less than ',
    43: ' plus ',
    38: ' and',
    37: 'percent',

}

hex_chars = ["7C", "7E", "22", "23", "27", "5B", "60", "5E"]

In [9]:
# ASCII character replacements

for ascii_code, replacement in ascii_list.items():
    ascii_char = chr(ascii_code)
    # Replace ASCII character with its equivalent
    data['description_clean'] = data['description_clean'].str.replace(ascii_char, replacement)
    
    # Escape the ASCII character
    escaped_ascii_char = re.escape(ascii_char)

    # Identify rows where the ASCII character was present
    modified_rows = data[data['description'].str.contains(escaped_ascii_char, na=False)]

    # Log the number of modified rows and a preview of modified rows
    num_modified = len(modified_rows)

    logging.info(f"Replacement for ASCII character {ascii_code} ({ascii_char}) with '{replacement}'")
    logging.info(f"Number of modified rows where ASCII character {ascii_code} was present: {num_modified}")
    if num_modified > 0:
        logging.info("Preview of modified rows:")
        preview = modified_rows.head(10)  # Preview first 10 modified rows
        logging.info(preview.to_string(index=False))


C:\Users\LatimerJ\AppData\Local\Temp\2\ipykernel_2904\2497773179.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['description_clean'] = data['description_clean'].str.replace(ascii_char, replacement)
C:\Users\LatimerJ\AppData\Local\Temp\2\ipykernel_2904\2497773179.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['description_clean'] = data['description_clean'].str.replace(ascii_char, replacement)
C:\Users\LatimerJ\AppData\Local\Temp\2\ipykernel_2904\2497773179.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=T

## Complex cleaning

In [10]:
# HEX char regex cleaning

# Replace the characters when they appear between two digits
for hex_char in hex_chars:
    char = bytes.fromhex(hex_char).decode('utf-8')
    # print(char)

    data['description_clean'] = data['description_clean'].str.replace(rf"(?<=\d){re.escape(char)}(?=\d)", "E", regex=True)

    # Identify rows where the HEX character was present
    modified_rows_hex = data[data['description'].str.contains(char, na=False, regex=False)]
    
    # Log the number of modified rows and a preview of modified rows
    num_modified_hex = len(modified_rows_hex)

    logging.info(f"Replacement for HEX character {char} with 'E'")
    logging.info(f"Number of modified rows where HEX character {char} was present: {num_modified_hex}")
    if num_modified_hex > 0:
        logging.info("Preview of modified rows:")
        preview = modified_rows_hex.head(10)  # Preview first 10 modified rows
        logging.info(preview.to_string(index=False))
    
    # adding a line here to clean up any remaining characters that were not between digits; inspect the output
    # to check that none of the hex_chars remain and that hex_chars between digits have correctly been replaced with "E"
    data['description_clean'] = data['description_clean'].str.replace(char, "")


    

C:\Users\LatimerJ\AppData\Local\Temp\2\ipykernel_2904\3244927265.py:25: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['description_clean'] = data['description_clean'].str.replace(char, "")
C:\Users\LatimerJ\AppData\Local\Temp\2\ipykernel_2904\3244927265.py:25: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['description_clean'] = data['description_clean'].str.replace(char, "")
C:\Users\LatimerJ\AppData\Local\Temp\2\ipykernel_2904\3244927265.py:25: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['description_cl

In [11]:
# Replace "?" between letters with " per "

char63 = bytes.fromhex("3F").decode('utf-8')


data['description_clean'] = data['description_clean'].str.replace(
    rf"(?<=[a-zA-Z]){re.escape(char63)}(?=[a-zA-Z])", " per ", regex=True)

# Log the number of modified rows and a preview of modified rows where "?" was present
modified_rows_char63 = data[data['description'].str.contains(re.escape(char63), na=False, regex=False)]
num_modified_char63 = len(modified_rows_char63)

logging.info(f"Replacement for HEX character {char63} with ' per '")
logging.info(f"Number of modified rows where HEX character {char63} was present: {num_modified_char63}")
if num_modified_char63 > 0:
    logging.info("Preview of modified rows:")
    preview = modified_rows_char63.head(10)  # Preview first 10 modified rows
    logging.info(preview.to_string(index=False))

# Remove any remaining "?" from description_clean
data['description_clean'] = data['description_clean'].str.replace(char63, "")

C:\Users\LatimerJ\AppData\Local\Temp\2\ipykernel_2904\2272361359.py:21: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['description_clean'] = data['description_clean'].str.replace(char63, "")


## More complex character cleaning

```
// hard coding
replace CleanDesc = subinstr(CleanDesc,  .)
replace CleanDesc = subinstr(CleanDesc, , .)

replace CleanDesc = subinstr(CleanDesc, , .)
replace CleanDesc = subinstr(CleanDesc, , .)
```

In [12]:
# char(47) "/" "\x2F"
char47 = bytes.fromhex("2F").decode('utf-8')

# digit then character then text
data['description_clean'] = data['description_clean'].str.replace(
    rf"(?<=\d){re.escape(char47)}(?=[a-zA-Z])", " per ", regex=True)
logging.info(f"Number of changes made by x2F replacement: {(data['description_clean'] != data['description']).sum()}")

# digit then character then text with space after digit
data['description_clean'] = data['description_clean'].str.replace(
    rf"(?<=\d\s){re.escape(char47)}(?=[a-zA-Z])", " per ", regex=True)
logging.info(f"Number of changes made by x2F replacement 2: {(data['description_clean'] != data['description']).sum()}")

# digit then character then text with space 
data['description_clean'] = data['description_clean'].str.replace(
    rf"(?<=\d\s){re.escape(char47)}(?=\s[a-zA-Z])", " per ", regex=True)
logging.info(f"Number of changes made by x2F replacement 3: {(data['description_clean'] != data['description']).sum()}")

# character surrounded by text
data['description_clean'] = data['description_clean'].str.replace(
    rf"(?<=[a-zA-Z]){re.escape(char47)}(?=[a-zA-Z])", " per ", regex=True)
logging.info(f"Number of changes made by x2F replacement 4: {(data['description_clean'] != data['description']).sum()}")

# text then character
data['description_clean'] = data['description_clean'].str.replace(
    rf"(?<=[a-zA-Z]){re.escape(char47)}", " per ", regex=True)
logging.info(f"Number of changes made by x2F replacement 5: {(data['description_clean'] != data['description']).sum()}")

# text then whitespace then character
data['description_clean'] = data['description_clean'].str.replace(
    rf"(?<=[a-zA-Z]\s){re.escape(char47)}", " per ", regex=True)
logging.info(f"Number of changes made by x2F replacement 6: {(data['description_clean'] != data['description']).sum()}")

# starts with character
data['description_clean'] = data['description_clean'].str.replace(
    rf"(^{re.escape(char47)})", " per ", regex=True)
logging.info(f"Number of changes made by x2F replacement 7: {(data['description_clean'] != data['description']).sum()}")

# starts with character followed by lower/ upper case L
data['description_clean'] = data['description_clean'].str.replace(
    rf"({re.escape(char47)}?=[l])", "per liter", regex=True)
data['description_clean'] = data['description_clean'].str.replace(
    rf"({re.escape(char47)}?=[L])", "per liter", regex=True)
logging.info(f"Number of changes made by x2F replacement 8: {(data['description_clean'] != data['description']).sum()}")

In [13]:
# hard coding

data['description_clean'] = data['description_clean'].str.replace("10/9L", "10E9L")
data['description_clean'] = data['description_clean'].str.replace("10/12L", "10E12L")
data['description_clean'] = data['description_clean'].str.replace("10/9 l", "10E9L")
data['description_clean'] = data['description_clean'].str.replace("10/12 l", "10E12L")
logging.info(f"Number of changes made by hard coding replacement: {(data['description_clean'] != data['description']).sum()}")

In [14]:
# char(32) " " \20 space
char32 = bytes.fromhex("20").decode('utf-8')
data['description_clean'] = data['description_clean'].str.replace(
    rf"(?<={re.escape(char32)}){re.escape(char32)}", "", regex=True)
logging.info(f"Number of changes made by char32 replacement: {(data['description_clean'] != data['description']).sum()}")

In [15]:
# char(46) "." \2E
char46 = bytes.fromhex("2E").decode('utf-8')

# replace ... with .
data['description_clean'] = data['description_clean'].str.replace(
    rf"(?<={re.escape(char46)}{re.escape(char46)}){re.escape(char46)}", "", regex=True)
logging.info(f"Number of changes made by char46 replacement 1: {(data['description_clean'] != data['description']).sum()}")

# remove character surrounded by text
data['description_clean'] = data['description_clean'].str.replace(
    rf"(?<=[a-zA-Z]){re.escape(char46)}(?=[a-zA-Z])", " ", regex=True)
logging.info(f"Number of changes made by char46 replacement 2: {(data['description_clean'] != data['description']).sum()}")

# remove character surrounded by space and text
data['description_clean'] = data['description_clean'].str.replace(
    rf"(?<=[a-zA-Z]\s){re.escape(char46)}(\s[a-zA-Z])", " ", regex=True)
logging.info(f"Number of changes made by char46 replacement 3: {(data['description_clean'] != data['description']).sum()}")

# remove character at the end of text
data['description_clean'] = data['description_clean'].str.replace(
    rf"(?<=[a-zA-Z]){re.escape(char46)}$", " ", regex=True)
logging.info(f"Number of changes made by char46 replacement 4: {(data['description_clean'] != data['description']).sum()}")


In [16]:
# char(45) "-" \2D
char45 = bytes.fromhex("2D").decode('utf-8')

# remove character surrounded by text
data['description_clean'] = data['description_clean'].str.replace(
    rf"(?<=[a-zA-Z]){re.escape(char45)}(?=[a-zA-Z])", " ", regex=True)
logging.info(f"Number of changes made by char45 replacement 1: {(data['description_clean'] != data['description']).sum()}")

# Remove hyphen surrounded by letters (convert hyphen to space)
data['description_clean'] = data['description_clean'].str.replace(
    rf"(?<=[a-zA-Z]){re.escape(char45)}(?=[a-zA-Z])", " ", regex=True)

# Remove hyphen surrounded by spaces and letters (convert hyphen and space to a single space)
data['description_clean'] = data['description_clean'].str.replace(
    rf"(?<=\s){re.escape(char45)}(?=\s)", "", regex=True)
data['description_clean'] = data['description_clean'].str.replace(
    rf"(?<=[a-zA-Z]\s){re.escape(char45)}(\s[a-zA-Z])", " ", regex=True)
data['description_clean'] = data['description_clean'].str.replace(
    rf"(?<=[a-zA-Z]\s){re.escape(char45)}([a-zA-Z])", " ", regex=True)
logging.info(f"Number of changes made by char45 replacement 2: {(data['description_clean'] != data['description']).sum()}")

# replace character with 'to' surrounded by digits
data['description_clean'] = data['description_clean'].str.replace(
    rf"(?<=\d){re.escape(char45)}(?=\d)", " to ", regex=True)
logging.info(f"Number of changes made by char45 replacement 2: {(data['description_clean'] != data['description']).sum()}")

# replace character with 'to' surrounded by digits and space
data['description_clean'] = data['description_clean'].str.replace(
    rf"(\d\s){re.escape(char45)}(\s\d)", " to ", regex=True)
logging.info(f"Number of changes made by char45 replacement 3: {(data['description_clean'] != data['description']).sum()}")

# remove char46 char45 surrounded by text
data['description_clean'] = data['description_clean'].str.replace(
    rf"(?<=[a-zA-Z]){re.escape(char46)}{re.escape(char45)}(?=[a-zA-Z])", " ", regex=True)
logging.info(f"Number of changes made by char45 replacement 4: {(data['description_clean'] != data['description']).sum()}")

# remove - at beginiing
data['description_clean'] = data['description_clean'].str.replace(
    rf"(^{re.escape(char45)})", "", regex=True)
logging.info(f"Number of changes made by char45 replacement 5: {(data['description_clean'] != data['description']).sum()}")

# remove - at end
data['description_clean'] = data['description_clean'].str.replace(
    rf"{re.escape(char45)}$", "", regex=True)
logging.info(f"Number of changes made by char45 replacement 6: {(data['description_clean'] != data['description']).sum()}")

# all else remove
data['description_clean'] = data['description_clean'].str.replace(char45, " ")
logging.info(f"Number of changes made by char45 replacement 7: {(data['description_clean'] != data['description']).sum()}")


In [17]:
# (more) hard coding

data['description_clean'] = data['description_clean'].str.replace("10-9 per l", "10E9 per l")
data['description_clean'] = data['description_clean'].str.replace("10-12 per l", "10E12 per l")
data['description_clean'] = data['description_clean'].str.replace("10-9 per L", "10E9 per L")
data['description_clean'] = data['description_clean'].str.replace("10-12 per L", "10E12 per L")
logging.info(f"Number of changes made by hard coding replacement 2: {(data['description_clean'] != data['description']).sum()}")

In [18]:
# remove char(239)
char239 = chr(239)
data['description_clean'] = data['description_clean'].str.replace(char239, "")
logging.info(f"Number of changes made by char(239) replacement: {(data['description_clean'] != data['description']).sum()}")

In [19]:
# Stage 2 parts per notation

data['description_clean'] = data['description_clean'].str.replace("10e9", " billion ")
data['description_clean'] = data['description_clean'].str.replace("10*9", " billion ")
data['description_clean'] = data['description_clean'].str.replace("10x9", " billion ")
data['description_clean'] = data['description_clean'].str.replace("10/9", " billion ")
data['description_clean'] = data['description_clean'].str.replace("10 9", " billion ")
data['description_clean'] = data['description_clean'].str.replace("10e6", " million ")
data['description_clean'] = data['description_clean'].str.replace("10x6", " million ")
data['description_clean'] = data['description_clean'].str.replace("10*6", " million ")
data['description_clean'] = data['description_clean'].str.replace("10/6", " million ")
data['description_clean'] = data['description_clean'].str.replace("10 6", " million ")
data['description_clean'] = data['description_clean'].str.replace("10e12", " trillion ")
data['description_clean'] = data['description_clean'].str.replace("10x12", " trillion ")
data['description_clean'] = data['description_clean'].str.replace("10/12", " trillion ")
data['description_clean'] = data['description_clean'].str.replace("10 12", " trillion ")
data['description_clean'] = data['description_clean'].str.replace("10e3", " thousand ")
data['description_clean'] = data['description_clean'].str.replace("10x3", " thousand ")
data['description_clean'] = data['description_clean'].str.replace("10 3", " thousand ")
data['description_clean'] = data['description_clean'].str.replace("10e7", " ten million ")
data['description_clean'] = data['description_clean'].str.replace("10x7", " ten million ")
data['description_clean'] = data['description_clean'].str.replace("8 x 10e8", " eight times 10 to the 8th ")
data['description_clean'] = data['description_clean'].str.replace("8x 10e8", " eight times 10 to the 8th ")
data['description_clean'] = data['description_clean'].str.replace("8x10e8", " eight times 10 to the 8th ")
data['description_clean'] = data['description_clean'].str.replace("10e8", " 10 to the 8th ")
data['description_clean'] = data['description_clean'].str.replace("10e5", " hundred thousand ")
data['description_clean'] = data['description_clean'].str.replace("10x5", " hundred thousand ")
data['description_clean'] = data['description_clean'].str.replace("10e4", " ten thousand ")
data['description_clean'] = data['description_clean'].str.replace("10x4", " ten thousand ")
data['description_clean'] = data['description_clean'].str.replace("10e2", " one hundred ")
data['description_clean'] = data['description_clean'].str.replace("10x2", " one hundred ")
data['description_clean'] = data['description_clean'].str.replace("10e10", " ten to the 10th power ")
data['description_clean'] = data['description_clean'].str.replace("10x10", " ten to the 10th power ")
logging.info(f"Number of changes made by stage 2 parts per notation replacement: {(data['description_clean'] != data['description']).sum()}")

In [20]:
# Stage 3 other spellling variations

data['description_clean'] = data['description_clean'].str.replace(
    rf"[Mm](th|onthly|onths|onth|\\.?)", "month", regex=True)
data['description_clean'] = data['description_clean'].str.replace(
    rf"[Yy](rly|rs|r|yyy|yy|y|ers|eras|ears|eears|\\.?)", "year", regex=True)
data['description_clean'] = data['description_clean'].str.replace(
    rf"[Ww](eek|eeks|ee|e|k|\\.?)", "week", regex=True)
data['description_clean'] = data['description_clean'].str.replace(
    rf"[Dd](ay|ays|y|\\.?)", "day", regex=True)
data['description_clean'] = data['description_clean'].str.replace(
    rf"[Ll](itre|iter|itres|iters|tr|\\.?)", "litre", regex=True)

In [21]:
# whitespace
whitespace_char = bytes.fromhex("20").decode('utf-8')

# remove whitespace at beginning and end
data['description_clean'] = data['description_clean'].str.replace(
    rf"(^{re.escape(whitespace_char)})", "", regex=True)
data['description_clean'] = data['description_clean'].str.replace(
    rf"({re.escape(whitespace_char)}$)", "", regex=True)
# replace two or more whitespace characters with one
df['description_clean'] = df['description_clean'].str.replace(
    rf'{re.escape(whitespace_char)}{{2,}}', whitespace_char, regex=True)
logging.info(f"Number of changes made by whitespace_char replacement: {(data['description_clean'] != data['description']).sum()}")


In [ ]:
# time unit measurements

data['description_clean'] = data['description_clean'].str.replace("daily", "per day")
data['description_clean'] = data['description_clean'].str.replace("a month", " per month")
data['description_clean'] = data['description_clean'].str.replace("a week", " per week")

In [22]:
data.to_csv('2024_03_numunit_withOMOPtarget_synonyms_stage2.csv', index=False)
print("Preprocessing completed")

Preprocessing completed


## Add mappings to `source_code_description`

In [ ]:
def replace_source_code_description(data, keyword, replacement):
    data.loc[data['description_clean'].str.contains(keyword, case=False), 'source_code_description'] = replacement

replace_source_code_description(data, 'hours', 'unit of time')
replace_source_code_description(data, 'million per ul', 'million per microliter')
replace_source_code_description(data, 'million per microl', 'million per microliter')


